In [1]:
import time
import pandas as pd
import json
from web3 import Web3

# Connect to ETH blockchain with infura API key
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/5f0c4998d7544ee1bb3f0dc297a6821c'))

# AAVE V3 ETH Mainnet Market Pool Address
contract_addr = '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2'

# Load ABI
with open("./abi.json") as f:
    pool_abi = json.load(f)
    
# Load Contract
aave_pool_contract = w3.eth.contract(address=contract_addr, abi=pool_abi)

In [2]:
# Loop from latest block and decrement by block_step until 10000 logs gathered

# Filter for withdrawls associated with rETH address as reserve
rETH_token_address = '0xae78736Cd615f374D3085123A210448E74Fc6393'

# Variables for looping
logged_withdrawls = []
target_log_count = 10000
current_block = w3.eth.block_number    # Latest Block Number
block_step = 500000

# Time
start = time.time()

# Loop until target number reached or at first block
while (len(logged_withdrawls) < target_log_count and current_block > 0):
    print(f'Current Block: {current_block}\nCurrent Log Count: {len(logged_withdrawls)}')
    from_block = max(current_block - block_step, 0)
    try:
        # Fetch logs for the current block
        current_logs = aave_pool_contract.events.Withdraw().get_logs(
            fromBlock=from_block,
            toBlock=current_block,
            argument_filters={'reserve': rETH_token_address}
        )
        
        # Format event logs
        for event in current_logs:
            formatted_log = {
                'Reserve': event['args']['reserve'].lower(),
                'User (Wallet Address)': event['args']['user'].lower(),
                'Amount (Wei)': event['args']['amount'],
                'Amount (ETH)': event['args']['amount'] / 10**18,
                'LogIndex': event['logIndex'],
                'TransactionIndex': event['transactionIndex'],
                'TransactionHash': event['transactionHash'].hex().lower(),
                'BlockHash': event['blockHash'].hex().lower(),
                'BlockNumber': event['blockNumber']
            }
       
            # Add to aggregate log list
            logged_withdrawls.append(formatted_log)

        # Decrement block number
        current_block = from_block - 1
        
    except Exception as e:
        # Print exception and block number and break
        print(f"Error fetching logs for block {current_block}: {e}")
        break
        
# Print time
print(f'Time Elapsed: {time.time() - start}')
       
# Trim list
if len(logged_withdrawls) > target_log_count:
       logged_withdrawls = logged_withdrawls[-target_log_count:]
        
rETH_withdrawls = pd.DataFrame(logged_withdrawls)
rETH_withdrawls

Current Block: 19158688
Current Log Count: 0
Current Block: 18658687
Current Log Count: 246
Current Block: 18158686
Current Log Count: 425
Current Block: 17658685
Current Log Count: 596
Current Block: 17158684
Current Log Count: 714
Current Block: 16658683
Current Log Count: 775
Current Block: 16158682
Current Log Count: 778
Current Block: 15658681
Current Log Count: 778
Current Block: 15158680
Current Log Count: 778
Current Block: 14658679
Current Log Count: 778
Current Block: 14158678
Current Log Count: 778
Current Block: 13658677
Current Log Count: 778
Current Block: 13158676
Current Log Count: 778
Current Block: 12658675
Current Log Count: 778
Current Block: 12158674
Current Log Count: 778
Current Block: 11658673
Current Log Count: 778
Current Block: 11158672
Current Log Count: 778
Current Block: 10658671
Current Log Count: 778
Current Block: 10158670
Current Log Count: 778
Current Block: 9658669
Current Log Count: 778
Current Block: 9158668
Current Log Count: 778
Current Block: 86

,Reserve,User (Wallet Address),Amount (Wei),Amount (ETH),LogIndex,TransactionIndex,TransactionHash,BlockHash,BlockNumber
0,0xae78736cd615f374d3085123a210448e74fc6393,0xb98a2d64bcf154238c3f2ac9922544ae0d4989d8,5000000000000000000,5.000000,254,126,0x1bad75b5b2e2a419de6aa4ef4dd04fb99039f28d4108...,0xa295a631fb4bdbaf3251faa4d2ac4e95019acc960980...,18659417
1,0xae78736cd615f374d3085123a210448e74fc6393,0x4fe23e87d977ea6484b01915ccd2746335e55dfc,35549611725958401562,35.549612,237,99,0x76eeba487e97220686552e8316bffa658d349ccf27ba...,0xd06e25e635a8c4b88574c2e0de24337f52e5c08ccb72...,18661218
2,0xae78736cd615f374d3085123a210448e74fc6393,0x684566c9ffcac7f6a04c3a9997000d2d58c00824,280834969005645292553,280.834969,169,120,0x6f9116d6ecb6354d89ed4c6332b47cde8b5804be2ae6...,0xb53f3a8b1687f4f35784bcc8d6572bdf86b136b021a6...,18676372
3,0xae78736cd615f374d3085123a210448e74fc6393,0xe4f646f0be4ff5ce185540f5366295f91d75b65d,18000000000000000000,18.000000,245,65,0xdc1acdba85464376ba072adb4e0d1a7f25e4195332a1...,0xe2920864ac9d729765122d2e5175a3074d7a9f410833...,18680353
4,0xae78736cd615f374d3085123a210448e74fc6393,0x02e7b8511831b1b02d9018215a0f8f500ea5c6b3,469992023707772084,0.469992,218,37,0xd6fd1356a3001b001355688a91f84e35320aed0f8bb7...,0x77d54dc1ffa1e0830fd6af02971f13259de442619a8b...,18681378
...,...,...,...,...,...,...,...,...,...
773,0xae78736cd615f374d3085123a210448e74fc6393,0xa21658e236ac33c0a23393bf889536f031f1e1ad,3000000000000000000,3.000000,155,75,0x56000379d19653a0a9fe5bdae1bf7e7700b3c946cf2c...,0x038005913e622e5d2bd40775cfb8f7d8861137dfa453...,17153960
774,0xae78736cd615f374d3085123a210448e74fc6393,0xa453e68f7208dd20d74fb0034073873ac04b017f,3453950767901774575,3.453951,261,230,0x2e9dfe43317e13ed3bfa2b8f7f3e314b63965b987e24...,0xb1f4e7cccacc6b0848ca9282ecb14bc51587eef029c6...,17154206
775,0xae78736cd615f374d3085123a210448e74fc6393,0xa59ea3b9fee7a1d1d9c533fe8ae215f97b8e054b,883321022717841974,0.883321,500,273,0x3e5c89715964584537d16c1b23bc5e710d3094c8f072...,0x2862ef4204d8e71115b999d734740c8246c90c3199be...,16628078
776,0xae78736cd615f374d3085123a210448e74fc6393,0x66ae11ebfcd932e02f4112aa1659c7c4caac863e,144675392009862883,0.144675,92,37,0x39c54f61920be6cfa35dd7fab50f643cd043f22933e6...,0xf26766a133fe9153689dcb01305520d480d26e35a32a...,16632976


In [3]:
# Catch withdrawl events from earliest to latest block
withdrawls = aave_pool_contract.events.Withdraw().get_logs(fromBlock='earliest',
                                                           toBlock='latest',
                                                           argument_filters={'reserve': '0xae78736Cd615f374D3085123A210448E74Fc6393'})

# Convert Transactions into ordinary dict format
withdraw_event_list = [
    {
        'Reserve': event['args']['reserve'].lower(),
        'User (Wallet Address)': event['args']['user'].lower(),
        'Amount (Wei)': event['args']['amount'],
        'Amount (rETH)': event['args']['amount']/10**18,
        'LogIndex': event['logIndex'],
        'TransactionIndex': event['transactionIndex'],
        'TransactionHash': event['transactionHash'].hex().lower(),
        'BlockHash': event['blockHash'].hex().lower(),
        'BlockNumber': event['blockNumber']
    }
    for event in withdrawls
]

# Push withdrawls into dataframe
total_withdrawls = pd.DataFrame(withdraw_event_list)
total_withdrawls

,Reserve,User (Wallet Address),Amount (Wei),Amount (ETH),LogIndex,TransactionIndex,TransactionHash,BlockHash,BlockNumber
0,0xae78736cd615f374d3085123a210448e74fc6393,0xa59ea3b9fee7a1d1d9c533fe8ae215f97b8e054b,883321022717841974,0.883321,500,273,0x3e5c89715964584537d16c1b23bc5e710d3094c8f072...,0x2862ef4204d8e71115b999d734740c8246c90c3199be...,16628078
1,0xae78736cd615f374d3085123a210448e74fc6393,0x66ae11ebfcd932e02f4112aa1659c7c4caac863e,144675392009862883,0.144675,92,37,0x39c54f61920be6cfa35dd7fab50f643cd043f22933e6...,0xf26766a133fe9153689dcb01305520d480d26e35a32a...,16632976
2,0xae78736cd615f374d3085123a210448e74fc6393,0x0d787fd9ff6a667f9722cf68574371e390e9ed40,23569197045276901856,23.569197,173,66,0x0561ec232b51eaa7d53f278e28eaabd7823f32f670e8...,0x3d47336d7a617f960be16351fa0567302cc0ebbdf834...,16651571
3,0xae78736cd615f374d3085123a210448e74fc6393,0xe7c5ff8b017f48a2275a3f9f4f01a1e664cb81ee,47582887547365515139,47.582888,304,145,0xa1af1dd872c6e977f60d72004d2bc6a7de6a23399e06...,0xe8b6890be4e3c169b03ed85db719a15b92eca385c274...,16709700
4,0xae78736cd615f374d3085123a210448e74fc6393,0xe45218b20ce500cd132bc6312f291827f3d6db32,7637357086490277567,7.637357,707,235,0xd51607db0d055237d3892b1f54c295f499ab52d0b4e2...,0xd4adfdab5d0fbd94d247b5c698c35f96a13a2f0e41ef...,16715126
...,...,...,...,...,...,...,...,...,...
773,0xae78736cd615f374d3085123a210448e74fc6393,0x713c89621b8a97b1c401793efc4796377bd55e49,6370000000000000000,6.370000,162,139,0x5a64550056b941d1199f76a93260a5fdd89ce0283590...,0x52ba747472bf41720932120e1f91ecaca7dc7d1b1424...,19154404
774,0xae78736cd615f374d3085123a210448e74fc6393,0x1aecea38b8626eeb3748234343cff427268dd487,896368585093886228144,896.368585,87,5,0xbf2f7d8e0edaaa0aa1acd8f672a0ee8e9a021a329321...,0xc6cfa05b1908cce44799e64cb7928db653fa906db61b...,19154481
775,0xae78736cd615f374d3085123a210448e74fc6393,0x5c93b495eebcf54d024401c70e6a3c72af93ca04,736421658699623954,0.736422,326,64,0x27c814547a7b408b964a9a9cbdfd35af78e307459493...,0x6239e9a785c3e180e19f062dd84fb681deec1ca45f5d...,19154528
776,0xae78736cd615f374d3085123a210448e74fc6393,0x1aecea38b8626eeb3748234343cff427268dd487,3933623053539381843265,3933.623054,80,2,0x8562fa6615871951b4ee88be17e53aef702b0d3c4701...,0x7214236ce0b63b4888281e5ff30645782f798fe3e306...,19154532


In [7]:
# Get transaction hash of first withdrawl on record
total_withdrawls.loc[0, 'TransactionHash']

'0x3e5c89715964584537d16c1b23bc5e710d3094c8f072f002d8e9876d23e1dd3b'